In [9]:
!pip install -U -q openai tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [10]:
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
import pandas as pd

In [11]:
# Read the OpenAI API key
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OpenAI_Api_Key')

In [47]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''
    system_message = f"""
    You are a knowledgeable assistant specialized in restaurant recommendations and table reservations. You understand various cuisines, dining preferences, and local restaurant options, tailored to the user’s city.
    Kindly ask the user to provide their dining requirements, including location and cuisine preferences, while providing some cusine preferences such as North Indian, Italian, Chinese, Mexican, etc..

    Once the user share there city and cusine preference, your goal is to fetch suggest the best 5 available restaurants that match their needs.

    After suggesting the restraunts, ask them to select one of the restaurants for making the reservation.
    Then, you must ask the user for their name and date time to confirm the table reservation.
    Once you have reservation details, you must reserve the table with provided details.
    """

    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [23]:
restaurants_df = pd.read_csv('restaurants_data.csv')
cuisines_list = []
for index, row in restaurants_df.iterrows():
  cuisines = row['Cuisines'].split(', ')
  for cuisine in cuisines:
    if cuisine not in cuisines_list:
      cuisines_list.append(cuisine)

print(cuisines_list)


['North Indian', 'Mughlai', 'Rajasthani', 'Chinese', 'European', 'Gujarati', 'Continental', 'South Indian', 'Desserts', 'Cafe', 'Italian', 'Mexican', 'Pizza', 'Fast Food', 'Mediterranean', 'Thai', 'Ice Cream', 'Beverages', 'Asian', 'Street Food', 'Sandwich', 'Burger', 'Healthy Food', 'American', 'Armenian', 'Salad', 'Bakery', 'Mithai', 'Biryani', 'Juices', 'Maharashtrian', 'Hyderabadi', 'Modern Indian', 'Finger Food', 'Tex-Mex', 'Arabian', 'Charcoal Grill', 'Steak', 'Seafood', 'Tea', 'Japanese', 'Malaysian', 'Burmese', 'Chettinad', 'Spanish', 'Greek', 'Indian', 'Parsi', 'Tibetan', 'Raw Meats', 'French', 'Goan', 'German', 'Kerala', 'Lebanese', 'Belgian', 'Kashmiri', 'Sushi', 'South American', 'Persian', 'Bengali', 'Portuguese', 'African', 'Iranian', 'Vietnamese', 'Lucknowi', 'Korean', 'Awadhi', 'Nepalese', 'Drinks Only', 'Pakistani', 'North Eastern', 'Oriya', 'Bihari', 'Afghani', 'Middle Eastern', 'Indonesian', 'Assamese', 'Andhra', 'Mangalorean', 'British', 'Malwani', 'Cuisine Varies',

In [16]:
def find_best_restaurants(city, cuisine = None):
  """
  Finds the top 5 restaurants based on aggregate ratings for a given city and cuisine.

  Args:
    city: The city to search for restaurants in.
    cuisine: The cuisine to search for.

  Returns:
    A list of dictionaries, where each dictionary represents a restaurant and contains
    its name, rating, and address.
  """
  try:
    # Load restaurants data from CSV file
    restaurants_df = pd.read_csv('restaurants_data.csv')

    # Filter restaurants by city and cuisine
    filtered_restaurants = restaurants_df[restaurants_df['City'].str.contains(city, case = False)]

    if cuisine:
      filtered_restaurants = filtered_restaurants[filtered_restaurants['Cuisines'].str.contains(cuisine, case=False)]

    # Sort restaurants by aggregate ratings in descending order
    sorted_restaurants = filtered_restaurants.sort_values('Aggregate rating', ascending=False)

    # Get the top 5 restaurants
    top_restaurants = sorted_restaurants.head(5)

    # Convert the top restaurants to a list of dictionaries
    restaurant_list = []
    for _, row in top_restaurants.iterrows():
      restaurant_list.append({
          'name': row['Restaurant Name'],
          'rating': row['Aggregate rating'],
          'address': row['Address'],
          'cuisines': row['Cuisines']
      })

    return restaurant_list

  except Exception as e:
    print(f"An error occurred: {e}")
    return []


In [29]:
eatMateFunctions = [
    {
      "name": "findBestRestaurants",
      "description": "Find the best 5 restaurants for a given cuisine in a city",
      "parameters": {
          "type": "object",
          "properties": {
              "city": {
                  "type": "string",
                  "description": "The city where user is looking for restaurants"
              },
              "cuisine": {
                  "type": "string",
                  "enum": cuisines_list,
                  "description": "cuisine prefered by user"
              }
          },
          "required": ["city"]
        }
    },
    {
        "name": "tableReservation",
        "description": "Reserve a table for the user at specific restaurant, at given date and time",
        "parameters": {
            "type": "object",
            "properties": {
                "restaurant": {
                    "type": "string",
                    "description": "The name of the restaurant"
                },
                "user_name": {
                    "type": "string",
                    "description": "The name of the user"
                },
                "date_time": {
                    "type": "string",
                    "description": "The date and time of the reservation"
                }
            },
            "required" : ["restaurant", "user_name", "date_time"]
            }
        }
    ]

In [14]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    chat_completion = openai.chat.completions.create(
        model = MODEL,
        messages = input,
        functions = eatMateFunctions,
        function_call = "auto",
        seed = 2345)

    output = chat_completion.choices[0]

    return output

In [15]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(content):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=content)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [17]:
find_best_restaurants("delhi", "north indian")

[{'name': 'Kopper Kadai',
  'rating': 4.8,
  'address': 'J2/6B, 1st & 2nd Floor, B.K. Dutta Market, Rajouri Garden, New Delhi',
  'cuisines': 'North Indian'},
 {'name': 'Zabardast Indian Kitchen',
  'rating': 4.7,
  'address': 'E-13/29, Ground Floor, Middle Circle, Connaught Place, New Delhi',
  'cuisines': 'North Indian'},
 {'name': 'Band Baaja Baaraat',
  'rating': 4.6,
  'address': 'A-6 Ground Floor, Vishal Enclave, Rajouri Garden, New Delhi',
  'cuisines': 'North Indian'},
 {'name': 'Food Scouts',
  'rating': 4.6,
  'address': 'Punjabi Bagh, New Delhi',
  'cuisines': 'North Indian, Chinese, Continental'},
 {'name': 'Qubitos - The Terrace Cafe',
  'rating': 4.5,
  'address': 'C-7, Vishal Enclave, Opposite Metro Pillar 417, Rajouri Garden, New Delhi',
  'cuisines': 'Thai, European, Mexican, North Indian, Chinese, Cafe'}]

In [66]:
def invoke_function(function_call):
  """
  Invokes a function based on the function_call object.
  """
  if function_call.name == "findBestRestaurants":
    arguments = json.loads(function_call.arguments)
    city = arguments.get("city")
    cuisine = arguments.get("cuisine")
    restaurants = find_best_restaurants(city, cuisine)
    return restaurants

  elif function_call.name == "tableReservation":
    arguments = json.loads(function_call.arguments)
    restaurant = arguments.get("restaurant")
    user_name = arguments.get("user_name")
    date_time = arguments.get("date_time")

    return (f"Reservation confirmed - Restaurant: {restaurant}, User Name: {user_name}, Date Time: {date_time}")

  else:
    raise ValueError(f"Unknown function: {function_call.name}")

In [75]:
from IPython.display import display, Markdown, Latex

def dialogue_mgmt_system():
    conversation = initialize_conversation()
    introduction = get_chat_completions(conversation).message.content

    display(Markdown(f"**EatMate:** {introduction}"))

    user_input = ''

    while(user_input != "exit"):
        display(Markdown("**You:** Please provide your input here or type 'exit' to finish:"))
        user_input = input()

        if(user_input == "exit"):
            break

        moderation = moderation_check(user_input)

        if moderation == 'Flagged':
            display(Markdown("Sorry, this message has been flagged. Please restart your conversation."))
            break

        conversation.append({"role": "user", "content": user_input})
        output = get_chat_completions(conversation)
        if output.message.function_call:
          func_resp = invoke_function(output.message.function_call)
          func_resp = json.dumps(func_resp)
          conversation.append({"role": "function", "name": output.message.function_call.name, "content": func_resp})
          output = get_chat_completions(conversation)
          output = output.message.content
        else:
          output = output.message.content

        display(Markdown(f"**EatMate:** {output}"))

        conversation.append({"role": "assistant", "content": output})

In [76]:
dialogue_mgmt_system()

**EatMate:** Hello! I can assist you with restaurant recommendations and table reservations. 
To get started, could you please provide me with your dining requirements? Specifically, let me know your city and cuisine preferences. Some cuisine options include North Indian, Italian, Chinese, Mexican, etc. 

May I know your city and preferred cuisine to help you find the best dining options?

**You:** Please provide your input here or type 'exit' to finish:

I want to have some chinese


**EatMate:** Sure, I can help you with that! Could you please tell me the name of your city so I can find the best Chinese restaurants for you?

**You:** Please provide your input here or type 'exit' to finish:

...

**You:** Please provide your input here or type 'exit' to finish:

thanks


**EatMate:** You're welcome! If you have any more questions or need assistance in the future, feel free to reach out. Enjoy your dining experience at Mumbai Vibe! Have a great day!

**You:** Please provide your input here or type 'exit' to finish:

exit
